In [1]:
import oatpy as oat

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import sklearn
import sklearn.metrics
from sklearn.neighbors import NearestNeighbors
import itertools

# Homology of a Dowker complex

In this example we will
- generate a point cloud
- generate a collection of subsets (some would call this a *cover*, others would call it a *hypergraph*)
- compute the homology of the associated Dowker complex
- analyze cycle representatives
- plot cycle representatives

## Generate a point cloud

In [2]:
#   SET PARAMETER VALUES
npoints             =   60
maxdis              =   None
maxdim              =   1

#   GENERATE CLOUD FROM RANDOM SEED
cloud               =   []
for seed in range(3):
    circle          =   oat.point_cloud.annulus(npoints=npoints, rad0=1, rad1=2.5, random_seed=seed)
    circle[:,0]     +=  2 * np.cos( seed * 2 * np.pi / 4)
    circle[:,1]     +=  2 * np.sin( seed * 2 * np.pi / 4)    
    cloud.append( circle )
cloud               =   np.concatenate(cloud)

dismat = sklearn.metrics.pairwise_distances(cloud)
dismat = ( dismat + dismat.T ) / 2

#   PLOT
trace = go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers")
fig = go.Figure([trace])
fig.update_layout( title=dict(text="Circle with noise"), width=800, height=800 )
fig.show()

## Choose a cover

In [3]:
radius_neighbor =   1.2; # hyperedges will be neighborhoods of vertices, with this radius
radius_net      =   1.0; # we'll make epsilon net with this value of epsilon

#   COMPUTE AN EPSILON NET
net, _          =   oat.dissimilarity.farthest_point_with_cloud( # the algorithm uses farthest point sampling
                        cloud       =   cloud, 
                        epsilon     =   radius_net 
                    ) 

#   PLOT THE COVER

data            =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True) )


for counter, v in enumerate(net):
    point = cloud[v]
    trace = oat.plot.ball_2d( point[0], point[1], radius=radius_neighbor, npoints=100 )
    trace.update( opacity=0.2, name=f"Cover {counter}" )
    data.append(trace)

fig = go.Figure(data)
fig.update_layout( title=dict(text="Hyperedges"), width=800, height=800 )
fig.show()

## Compute homology

We'll compute the homology of
- the dual hypergraph; that is, the hypergraph where vertices are balls, and for each vertex `v` we have a hyperedge that contains every ball to which `v` belongs
- equivalently, the witness complex where every point is a witness and net points are landmarks

The homology solver only accepts hypergraphs represented by a list of lists of integers, currently.  If your hypergraph has a different format (e.g., if vertices are strings), then you can use some built-in tools to help translate back and forth between this format and list-of-list format; see the [rbs_reduced](rbs_reduced.ipynb) notebook for examples.

In [4]:
#   FORMAT THE COVER AS A FAMILY OF SORTED LISTS

# data structure holding the whole cloud
net_wrapper         =   NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit( cloud[net] ) 
# witness complex where all points are witnesses, and net points are landmarks
cover               =   [ sorted(list(x)) for x in net_wrapper.radius_neighbors( cloud, radius=radius_neighbor, return_distance=False, ) ]    

#   FACTOR THE BOUNDARY MATRIX

factored            =   oat.rust.FactoredBoundaryMatrixDowker( 
                            dowker_simplices            =   cover, 
                            max_homology_dimension      =   2
                        )

#   BETTI STATISTICS

print("\n\nBetti statistics\n")
display( factored.betti() )

#   HOMOLOGY

print("\nHomology\n")
homology    =   factored.homology(                         
                return_cycle_representatives    =   True,  
                return_bounding_chains          =   True,  
            )
display( homology )

#   CYCLE REPRESENTATIVE

print("\nCycle representative\n")
display( homology["cycle representative"][1] )




Betti statistics



homology  space of chains  space of cycles  space of boundaries
dimension                                                                 
0                 1               27               27                   26
1                 4               60               34                   30
2                 0               34                4                    4


Homology



dimension birth simplex                               cycle representative   
0          0           [0]          simplex coefficient
0     [0]           1  \
1          1       [9, 25]     simplex coefficient
0  [9, 25]           1
...   
2          1       [8, 15]     simplex coefficient
0  [8, 15]           1
...   
3          1       [7, 16]      simplex coefficient
0   [7, 16]           ...   
4          1       [7, 15]      simplex coefficient
0   [7, 15]           ...   

   nnz  
0    1  
1    3  
2    7  
3   16  
4   12


Cycle representative



simplex coefficient
0  [9, 25]           1
1  [0, 25]          -1
2   [0, 9]           1

## Plot cycles

In [5]:
#   DATA FOR THE POINT CLOUD

data                    =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers+text", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True, text=[str(x) for x in range(len(net))],  textposition='bottom center',  ) )

#   DATA FOR THE CYCLE

coo                     =   cloud[net] # coo stands for "coordinate oracle"
colors                  =   px.colors.qualitative.Plotly # specifies a (discrete) sequence of colors, represented by a list of strings

for rownum, row in homology.iterrows():
    if row["dimension"] != 1: continue
    cycle_color         =   colors[ rownum % len(colors) ]

    for entrynum, entry in row["cycle representative"].iterrows():
        edge            =   entry["simplex"]
        coefficient     =   entry["coefficient"]

        trace           =   oat.plot.edge__trace2d( edge=edge, coo=coo )
        trace.update( name=f"Cycle {rownum}", text=f"simplex {edge}<br>linear coefficent {coefficient}", opacity=0.7, line=dict(color=cycle_color,)) # customize appearance
        trace.update( legendgroup=rownum) # group edges that belong to the same cycle
        trace.update( showlegend = entrynum==0 )
        data.append(trace) # append to the data group

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Coefficients still appear in the hover data", height=700,width=800 ) 
fig.show()

# Optimize

#### Within homology class

This method will find a minimal cycle representative within the same homology class

In [6]:
optimal     =   factored.optimize_cycle(
                    birth_simplex=[7,15], 
                    problem_type="preserve homology class"
                )

display(optimal)

display(optimal["chain"]["initial cycle"])

cost  nnz   
type of chain                               
initial cycle                   12.0   12  \
optimal cycle                   10.0   10   
difference in bounding chains    NaN    4   
difference in essential chains   NaN    6   
Ax + z - y                       NaN    6   

                                                                            chain  
type of chain                                                                      
initial cycle                       simplex coefficient
0   [7, 15]           ...  
optimal cycle                       simplex coefficient
0  [15, 26]          -...  
difference in bounding chains          simplex coefficient
0  [7, 15, 26]     ...  
difference in essential chains      simplex coefficient
0  [15, 26]          -...  
Ax + z - y                          simplex coefficient
0  [15, 26]          -...

simplex coefficient
0   [7, 15]           1
1   [7, 13]          -1
2   [6, 14]           1
3    [6, 8]          -1
4   [4, 17]          -1
5   [4, 11]           1
6   [3, 11]          -1
7    [3, 8]           1
8   [2, 14]          -1
9   [2, 13]           1
10  [0, 17]           1
11  [0, 15]          -1

In [7]:
#   DATA FOR THE POINT CLOUD

data                    =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers+text", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True, text=[str(x) for x in range(len(net))],  textposition='bottom center',  ) )

#   DATA FOR THE CYCLE

coo                     =   cloud[net] # coo stands for "coordinate oracle"
colors                  =   px.colors.qualitative.Plotly # specifies a (discrete) sequence of colors, represented by a list of strings

for cycle_num, cycle_name in enumerate(["optimal cycle", "initial cycle"]):

    cycle_color         =   colors[ cycle_num % len(colors) ]

    for entrynum, entry in optimal["chain"][cycle_name].iterrows():
        edge            =   entry["simplex"]
        coefficient     =   entry["coefficient"]

        trace           =   oat.plot.edge__trace2d( edge=edge, coo=coo )
        trace.update( name=cycle_name, text=f"simplex {edge}<br>linear coefficent {coefficient}", opacity=0.7, line=dict(color=cycle_color,)) # customize appearance
        trace.update( legendgroup=cycle_num) # group edges that belong to the same cycle
        trace.update( showlegend = entrynum==0 )
        data.append(trace) # append to the data group

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Initial and optimal cycles", height=700,width=800 ) 
fig.show()

#### Leaving homology class

The following method allows us to optimize the cycle representative by adding not only boundaries, but cycle represntatives for other homology classes.

In [8]:
optimal = factored.optimize_cycle(birth_simplex=[7,15], problem_type="preserve homology basis (once)")

display(optimal)

cost  nnz   
type of chain                               
initial cycle                   12.0   12  \
optimal cycle                    4.0    4   
difference in bounding chains    NaN    7   
difference in essential chains   NaN   16   
Ax + z - y                       NaN   11   

                                                                            chain  
type of chain                                                                      
initial cycle                       simplex coefficient
0   [7, 15]           ...  
optimal cycle                       simplex coefficient
0  [15, 26]          -...  
difference in bounding chains          simplex coefficient
0  [8, 15, 22]     ...  
difference in essential chains       simplex coefficient
0   [15, 26]         ...  
Ax + z - y                           simplex coefficient
0   [15, 26]         ...

In [9]:
#   DATA FOR THE POINT CLOUD

data                    =   []
data.append( go.Scatter(x=cloud[:,0],y=cloud[:,1], mode="markers", name="Cloud", showlegend=True)                                               )
data.append( go.Scatter(x=cloud[net,0],y=cloud[net,1], mode="markers+text", marker=dict(symbol="triangle-up", size=10), name="Net", showlegend=True, text=[str(x) for x in range(len(net))],  textposition='bottom center',  ) )

#   DATA FOR THE CYCLE

coo                     =   cloud[net] # coo stands for "coordinate oracle"
colors                  =   px.colors.qualitative.Plotly # specifies a (discrete) sequence of colors, represented by a list of strings

for cycle_num, cycle_name in enumerate(["optimal cycle", "initial cycle"]):

    cycle_color         =   colors[ cycle_num % len(colors) ]

    for entrynum, entry in optimal["chain"][cycle_name].iterrows():
        edge            =   entry["simplex"]
        coefficient     =   entry["coefficient"]

        trace           =   oat.plot.edge__trace2d( edge=edge, coo=coo )
        trace.update( name=cycle_name, text=f"simplex {edge}<br>linear coefficent {coefficient}", opacity=0.7, line=dict(color=cycle_color,)) # customize appearance
        trace.update( legendgroup=cycle_num) # group edges that belong to the same cycle
        trace.update( showlegend = entrynum==0 )
        data.append(trace) # append to the data group

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Initial and optimal cycles", height=700,width=800 ) 
fig.show()